# Tempo - Spark

## Preparando o ambiente

In [6]:
from pyspark.sql import SparkSession
import glob

In [8]:
spark = SparkSession\
    .builder\
    .appName("tempo")\
    .getOrCreate()

df = spark.read.csv(
    glob.glob("../data/tempo/semana=*/dados_brutos.csv"), 
    header=True, 
    inferSchema=True)
df.show(5)

+------+----------+-------+-------+----+------------+------------+---------+-----+--------+------+----------+-----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+-------------------+-------------------+---------+--------------------+--------------------+-----------------+--------------------+
|  name|  datetime|tempmax|tempmin|temp|feelslikemax|feelslikemin|feelslike|  dew|humidity|precip|precipprob|precipcover|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|            sunrise|             sunset|moonphase|          conditions|         description|             icon|            stations|
+------+----------+-------+-------+----+------------+------------+---------+-----+--------+------+----------+-----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-----

In [9]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- datetime: date (nullable = true)
 |-- tempmax: double (nullable = true)
 |-- tempmin: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- feelslikemax: double (nullable = true)
 |-- feelslikemin: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: integer (nullable = true)
 |-- precipcover: double (nullable = true)
 |-- preciptype: string (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- sealevelpressure: double (nullable = true)
 |-- cloudcover: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- uvindex: integ